In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.datasets as datasets
import torchvision.transforms as transforms



In [2]:

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomRotation(30),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

train_data_dir = "data/train"
train_dataset = datasets.ImageFolder(train_data_dir, transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=64, shuffle=True, num_workers=4)

val_data_dir = "data/val"
val_dataset = datasets.ImageFolder(val_data_dir, transform=val_transform)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=32, shuffle=False, num_workers=4)


print(train_dataset)





Dataset ImageFolder
    Number of datapoints: 93134
    Root location: data/train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.2)
               RandomRotation(degrees=[-30.0, 30.0], interpolation=nearest, expand=False, fill=0)
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [23]:
data = train_loader.dataset
print(data[0][0])
# use torchtoPIL to convert the tensor to a PIL Image
img = transforms.ToPILImage()(data[445][0]) 
img.show()


tensor([[[-0.0458, -0.0458, -0.0458,  ...,  0.0227,  0.0227,  0.0227],
         [-0.0458, -0.0458, -0.0458,  ...,  0.0398,  0.0398,  0.0398],
         [-0.0458, -0.0458, -0.0458,  ...,  0.0398,  0.0398,  0.0398],
         ...,
         [-1.8782, -1.8782, -1.8782,  ..., -0.8164, -0.6281, -0.4397],
         [-1.8782, -1.8782, -1.8782,  ..., -1.1247, -0.9363, -0.6965],
         [-1.8782, -1.8782, -1.8782,  ..., -1.3815, -1.1932, -0.9534]],

        [[ 0.1176,  0.1176,  0.1176,  ...,  0.1877,  0.1877,  0.1877],
         [ 0.1176,  0.1176,  0.1176,  ...,  0.2052,  0.2052,  0.2052],
         [ 0.1176,  0.1176,  0.1176,  ...,  0.2052,  0.2052,  0.2052],
         ...,
         [-1.7906, -1.7906, -1.7906,  ..., -0.6877, -0.4951, -0.3025],
         [-1.7906, -1.7906, -1.7906,  ..., -1.0028, -0.8102, -0.5826],
         [-1.7906, -1.7906, -1.7906,  ..., -1.2654, -1.0903, -0.8452]],

        [[ 0.2871,  0.2871,  0.2871,  ...,  0.3568,  0.3568,  0.3742],
         [ 0.2871,  0.2871,  0.2871,  ...,  0

In [3]:
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights= None)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', weights= None)
# generate random seed
seed = random.randint(0, 1000000)
# set the seed
print("Seed: ", seed)
torch.manual_seed(seed)

class AlexNet(nn.Module): 
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            # nn.Dropout(0.3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            # nn.Dropout(0.3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.BatchNorm2d(384,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            # nn.Dropout(0.3),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            # nn.Dropout(0.3),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            # nn.Dropout(0.3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.BatchNorm2d(256,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
model = AlexNet()
# open all layers
for param in model.parameters():
    param.requires_grad = True
# change the last layer
# model.fc = nn.Sequential(
#     nn.Linear(512, 1000),
#     nn.ReLU(),
#     nn.Dropout(0.2),
#     nn.Linear(1000, 1000),
#     nn.LogSoftmax(dim=1)
# )
print(model)


Seed:  908585
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
  

In [7]:
# set the seed
seed = random.randint(0, 1000000)
print("Seed: ", seed)
torch.manual_seed(seed)
# Resnet
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights= None)

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Using cache found in C:\Users\SWORWOOD/.cache\torch\hub\pytorch_vision_v0.10.0


In [4]:
# load the model
model = torch.load("best_model_0.359.pt")

In [8]:
import tqdm
# cuda settings
torch.cuda.empty_cache()  # 清空GPU缓存
torch.backends.cudnn.benchmark = True  # 启用自动寻找最适合当前配置的高效算法
torch.backends.cudnn.enabled = True  # 启用cudnn加速
# training
batch_size = 500
learning_rate = 0.00005
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()
train_loss = []
val_loss = []
train_acc = []
val_acc = []
epochs = 50
best_acc = 0
last_file = ''
# train in cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
for epoch in range(epochs):
    running_loss = 0.0
    running_acc = 0.0
    for i, data in tqdm.tqdm(enumerate(train_loader, 0), total=len(train_loader)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        running_acc += (predicted == labels).sum().item()

        
    running_loss /= len(train_loader)
    running_acc /= len(train_dataset)
    train_loss.append(running_loss)
    train_acc.append(running_acc)
    print(f"Epoch {epoch+1} train loss: {running_loss:.4f} train acc: {running_acc:.4f}")
    # validation
    model.eval()
    val_running_loss = 0.0
    val_running_acc = 0.0
    with torch.no_grad():
        for i, data in enumerate(val_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_running_acc += (predicted == labels).sum().item()
    val_running_loss /= len(val_loader)
    val_running_acc /= len(val_dataset)
    val_loss.append(val_running_loss)
    val_acc.append(val_running_acc)
    print(f"Epoch {epoch+1} val loss: {val_running_loss:.4f} val acc: {val_running_acc:.4f}")
    model.train()
    # save best model
    if val_running_acc > best_acc:
        best_acc = val_running_acc
        torch.save(model, 'best_model_'+str(best_acc)+'.pt')
        if last_file != '':
            os.remove(last_file)
        last_file = 'best_model_'+str(best_acc)+'.pt'
    torch.save(model, 'model_checkpoint.pt')

100%|██████████| 1456/1456 [01:34<00:00, 15.48it/s]


Epoch 1 train loss: 6.4892 train acc: 0.0110
Epoch 1 val loss: 6.2070 val acc: 0.0170


100%|██████████| 1456/1456 [01:32<00:00, 15.67it/s]


Epoch 2 train loss: 5.7371 train acc: 0.0353
Epoch 2 val loss: 5.7015 val acc: 0.0250


100%|██████████| 1456/1456 [01:33<00:00, 15.64it/s]


...

100%|██████████| 1456/1456 [01:33<00:00, 15.63it/s]


Epoch 46 train loss: 0.4897 train acc: 0.8813
Epoch 46 val loss: 1.8752 val acc: 0.5945


100%|██████████| 1456/1456 [01:33<00:00, 15.64it/s]


Epoch 47 train loss: 0.4661 train acc: 0.8869
Epoch 47 val loss: 1.8956 val acc: 0.5980


 21%|██        | 305/1456 [00:19<01:13, 15.65it/s]

In [11]:
class dataSet():
    def __init__(self):
        self.X_train = []
        self.y_train = []
        self.X_test = []
        self.y_test = []
        self.X_val = []
        self.y_val = []
        self.glo_mean = [] #for example [0.485, 0.456, 0.406] need to be calculated by the dataset
        self.glo_std = []  #for example [0.229, 0.224, 0.225] need to be calculated by the dataset

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        if not os.path.exists('DataSet'):
            os.makedirs('DataSet')

        if not os.path.exists('DataSet/X_train.pt'):
            # generate basic train, test, val data and also the one hot label
            self.load_data()
            # generate mean and std
            self.generate_mean_std()
            # normalize the data from 0-255 to 0-1
            self.normalize_data()
            # zero expand the data from 3*64*64 to 3*224*224
            self.zero_expand_data()
            # save the data to DataSet folder
            self.save_data()

        self.X_train = torch.load('DataSet/X_train.pt').to(device)
        self.y_train = torch.load('DataSet/y_train.pt').to(device)
        self.X_test = torch.load('DataSet/X_test.pt').to(device)
        self.y_test = torch.load('DataSet/y_test.pt').to(device)
        self.X_val = torch.load('DataSet/X_val.pt').to(device)
        self.y_val = torch.load('DataSet/y_val.pt').to(device)
        self.glo_mean = torch.load('DataSet/glo_mean.pt').to(device)
        self.glo_std = torch.load('DataSet/glo_std.pt').to(device)




    def prediction_accuracy(self, y_pred, y_true):
        return sum(y_pred == y_true) / len(y_true)
    
    def one_hot_encode(self, label):
        n = len(label)
        one_hot_label = np.zeros((n, n))
        for i in range(n):
            # a1_xxx -> xxx
            hot = int(label[i].split('_')[1])
            one_hot_label[i][hot] = 1
        return one_hot_label
    
    def load_data(self):
        current_dir = os.getcwd()
        train_list = os.listdir(current_dir + '/data/train')
        # Total classes
        n = len(train_list)
        # pic size 64*64*3
        label = train_list
        one_hot_label = self.one_hot_encode(label)
        # sort by one_hot_label.argmax() zip with label
        label = [x for _, x in sorted(zip(one_hot_label.argmax(axis=1), label))]
        one_hot_label = self.one_hot_encode(label)

        l = []
        for i in label:
            n = i.split('_')[1]
            l.append(one_hot_label[int(n)][int(n)]==1)
        # test if the one hot label is correct
        assert all(l)

        for i in train_list:
            pic_list = os.listdir(current_dir + '/data/train/' + i)
            for j in pic_list:
                img = Image.open(current_dir + '/data/train/' + i + '/' + j)
                img = np.array(img)
                img = torch.tensor(img)
                img = img.permute(2, 0, 1)
                img = img.unsqueeze(0).float()
                self.X_train.append(img)
                self.y_train.append(one_hot_label[train_list.index(i)])
        self.X_train = torch.cat(self.X_train)
        self.y_train = torch.tensor(self.y_train)
        self.y_train = self.y_train.argmax(1)
        # test and val data

        val_list = os.listdir(current_dir + '/data/val')
        for i in val_list:
            pic_list = os.listdir(current_dir + '/data/val/' + i)
            for j in pic_list:
                img = Image.open(current_dir + '/data/val/' + i + '/' + j)
                img = np.array(img)
                img = torch.tensor(img)
                img = img.permute(2, 0, 1)
                img = img.unsqueeze(0).float()
                self.X_val.append(img)
                self.y_val.append(one_hot_label[train_list.index(i)])
        self.X_val = torch.cat(self.X_val)
        self.y_val = torch.tensor(self.y_val)   
        self.y_val = self.y_val.argmax(1)

        test_list = os.listdir(current_dir + '/data/test')
        for i in test_list:
            pic_list = os.listdir(current_dir + '/data/test/' + i)
            for j in pic_list:
                img = Image.open(current_dir + '/data/test/' + i + '/' + j)
                img = np.array(img)
                img = torch.tensor(img)
                img = img.permute(2, 0, 1)
                img = img.unsqueeze(0).float()
                self.X_test.append(img)
                self.y_test.append(one_hot_label[train_list.index(i)])
        self.X_test = torch.cat(self.X_test)
        self.y_test = torch.tensor(self.y_test)
        self.y_test = self.y_test.argmax(1)
    
    def generate_mean_std(self):
        self.glo_mean = torch.mean(self.X_train, dim=(0, 2, 3))
        self.glo_std = torch.std(self.X_train, dim=(0, 2, 3))





    

            
    

In [12]:
DS = dataSet()
DS.load_data()

C:\Users\SWORWOOD\AppData\Local\Temp\ipykernel_6256\229839884.py:84: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  self.y_train = torch.tensor(self.y_train)


tensor([  0,   0,   0,  ..., 999, 999, 999])


AttributeError: 'dataSet' object has no attribute 'preprocess_data'

In [24]:
# test torch.mean(self.X_train, dim=(0, 2, 3)) and torch.std(self.X_train, dim=(0, 2, 3)) self.X_train is a n*3*64*64 tensor
# n is the number of pictures
current_dir = os.getcwd()
X_val = []
train_list = os.listdir(current_dir + '/data/train')
val_list = os.listdir(current_dir + '/data/val')
for i in val_list:
    pic_list = os.listdir(current_dir + '/data/val/' + i)
    for j in pic_list:
        img = Image.open(current_dir + '/data/val/' + i + '/' + j)
        img = np.array(img)
        img = torch.tensor(img)
        img = img.permute(2, 0, 1)
        img = img.unsqueeze(0).float()
        X_val.append(img)
X_val = torch.cat(X_val)
print(X_val.shape)

# init channel sum and pixel sum
channel_sum = [0, 0, 0]
channel_squared_sum = [0, 0, 0]
pixel_sum = 64*64*len(X_val)
for i in X_val:
    channel_sum += torch.sum(i, dim=(1, 2)).tolist()
    channel_squared_sum += torch.sum(i**2, dim=(1, 2)).tolist()
channel_mean = [i / pixel_sum for i in channel_sum]
channel_std = [np.sqrt(i / pixel_sum - j**2) for i, j in zip(channel_squared_sum, channel_mean)]

torch.Size([2000, 3, 64, 64])
[0, 0, 0, 265541.0, 265414.0, 233293.0, 62203.0, 63304.0, 79953.0, 242942.0, 243833.0, 125092.0, 209958.0, 207115.0, 92151.0, 139143.0, 126088.0, 77472.0, 88548.0, 87541.0, 97774.0, 305799.0, 313239.0, 319700.0, 194565.0, 176934.0, 119390.0, 347494.0, 362106.0, 425449.0, 210262.0, 205815.0, 230781.0, 178081.0, 190511.0, 134656.0, 179588.0, 164064.0, 255590.0, 296598.0, 261629.0, 194381.0, 169033.0, 174913.0, 145491.0, 281563.0, 280007.0, 268081.0, 136796.0, 115263.0, 104578.0, 204120.0, 197565.0, 148184.0, 113362.0, 115134.0, 142051.0, 115976.0, 107927.0, 173868.0, 234462.0, 204142.0, 153202.0, 194269.0, 189529.0, 145925.0, 160618.0, 138900.0, 93577.0, 189053.0, 179480.0, 153141.0, 143517.0, 126748.0, 96751.0, 268327.0, 253309.0, 162615.0, 497403.0, 515026.0, 510414.0, 458310.0, 475989.0, 514394.0, 240090.0, 238276.0, 236160.0, 160529.0, 148711.0, 108252.0, 69914.0, 65450.0, 63266.0, 254485.0, 215068.0, 179875.0, 428192.0, 425366.0, 248010.0, 116918.0, 110